# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [5]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [93]:
data.head()

CustomerID FirstName   LastName  SalesID  ProductID           ProductName  \
0       61288      Rosa   Andersen   134196        229  Bread - Hot Dog Buns   
1       77352     Myron     Murray  6167892        229  Bread - Hot Dog Buns   
2       40094     Susan  Stevenson  5970885        229  Bread - Hot Dog Buns   
3       23548    Tricia    Vincent  6426954        229  Bread - Hot Dog Buns   
4       78981     Scott      Burch   819094        229  Bread - Hot Dog Buns   

   Quantity  
0        16  
1        20  
2        11  
3         6  
4        20

## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [25]:
quantities = data.groupby(['CustomerID', 'ProductName']).agg({'Quantity': 'sum'})
quantities

Quantity
CustomerID ProductName                            
33         Apricots - Dried                      1
           Assorted Desserts                     1
           Bandage - Flexible Neon               1
           Bar Mix - Pina Colada, 355 Ml         1
           Beans - Kidney, Canned                1
...                                            ...
98200      Vol Au Vents                         50
           Wasabi Powder                        25
           Wine - Fume Blanc Fetzer             25
           Wine - Hardys Bankside Shiraz        25
           Yogurt - French Vanilla              25

[63628 rows x 1 columns]

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [14]:
table = pd.pivot_table(quantities, values='Quantity', index=['ProductName'],columns=['CustomerID'], aggfunc='sum')
table = table.fillna(0)
table

CustomerID                         33     200    264    356    412    464    \
ProductName                                                                   
Anchovy Paste - 56 G Tube            0.0    0.0    0.0    0.0    0.0    0.0   
Appetizer - Mini Egg Roll, Shrimp    0.0    0.0    0.0    0.0    0.0    0.0   
Appetizer - Mushroom Tart            0.0    0.0    0.0    0.0    0.0    0.0   
Appetizer - Sausage Rolls            0.0    0.0    0.0    0.0    0.0    0.0   
Apricots - Dried                     1.0    0.0    0.0    0.0    1.0    0.0   
...                                  ...    ...    ...    ...    ...    ...   
Yeast Dry - Fermipan                 0.0    0.0    0.0    0.0    0.0    0.0   
Yoghurt Tubes                        0.0    0.0    0.0    0.0    0.0    0.0   
Yogurt - Blueberry, 175 Gr           0.0    1.0    0.0    0.0    0.0    0.0   
Yogurt - French Vanilla              1.0    0.0    0.0    1.0    0.0    0.0   
Zucchini - Yellow                    0.0    0.0    0.0    0.0    0.0    0.0   

CustomerID                         477    639    649    669    ...  97697  \
ProductName                                                    ...          
Anchovy Paste - 56 G Tube            0.0    1.0    0.0    0.0  ...    0.0   
Appetizer - Mini Egg Roll, Shrimp    0.0    0.0    0.0    0.0  ...   25.0   
Appetizer - Mushroom Tart            0.0    1.0    0.0    0.0  ...   25.0   
Appetizer - Sausage Rolls            0.0    0.0    0.0    0.0  ...    0.0   
Apricots - Dried                     0.0    0.0    0.0    0.0  ...    0.0   
...                                  ...    ...    ...    ...  ...    ...   
Yeast Dry - Fermipan                 0.0    0.0    0.0    0.0  ...    0.0   
Yoghurt Tubes                        0.0    0.0    2.0    0.0  ...    0.0   
Yogurt - Blueberry, 175 Gr           0.0    0.0    0.0    0.0  ...   25.0   
Yogurt - French Vanilla              2.0    0.0    0.0    1.0  ...    0.0   
Zucchini - Yellow                    0.0    0.0    0.0    0.0  ...    0.0   

CustomerID                         97753  97769  97793  97900  97928  98069  \
ProductName                                                                   
Anchovy Paste - 56 G Tube           25.0    0.0    0.0    0.0    0.0    0.0   
Appetizer - Mini Egg Roll, Shrimp   25.0    0.0    0.0    0.0    0.0    0.0   
Appetizer - Mushroom Tart            0.0    0.0    0.0    0.0    0.0    0.0   
Appetizer - Sausage Rolls            0.0    0.0    0.0   25.0   25.0   25.0   
Apricots - Dried                    25.0    0.0    0.0    0.0    0.0    0.0   
...                                  ...    ...    ...    ...    ...    ...   
Yeast Dry - Fermipan                 0.0    0.0   25.0    0.0    0.0    0.0   
Yoghurt Tubes                        0.0    0.0    0.0    0.0    0.0    0.0   
Yogurt - Blueberry, 175 Gr           0.0    0.0   25.0    0.0    0.0    0.0   
Yogurt - French Vanilla              0.0   25.0    0.0    0.0    0.0    0.0   
Zucchini - Yellow                    0.0    0.0    0.0    0.0    0.0    0.0   

CustomerID                         98159  98185  98200  
ProductName                                             
Anchovy Paste - 56 G Tube            0.0    0.0    0.0  
Appetizer - Mini Egg Roll, Shrimp    0.0    0.0    0.0  
Appetizer - Mushroom Tart            0.0   25.0    0.0  
Appetizer - Sausage Rolls            0.0   25.0    0.0  
Apricots - Dried                     0.0    0.0    0.0  
...                                  ...    ...    ...  
Yeast Dry - Fermipan                 0.0    0.0    0.0  
Yoghurt Tubes                       25.0    0.0    0.0  
Yogurt - Blueberry, 175 Gr           0.0    0.0    0.0  
Yogurt - French Vanilla              0.0    0.0   25.0  
Zucchini - Yellow                    0.0    0.0    0.0  

[452 rows x 1000 columns]

## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [17]:
from scipy.spatial.distance import pdist, squareform

squareform(pdist(table.T, 'euclidean'))
distances = pd.DataFrame(1/(1 + squareform(pdist(table.T, 'euclidean'))), index=table.columns, columns=table.columns)
distances

CustomerID     33        200       264       356       412       464    \
CustomerID                                                               
33          1.000000  0.077421  0.087047  0.081800  0.080634  0.082709   
200         0.077421  1.000000  0.078448  0.076435  0.073693  0.075255   
264         0.087047  0.078448  1.000000  0.080070  0.081800  0.080350   
356         0.081800  0.076435  0.080070  1.000000  0.076435  0.078187   
412         0.080634  0.073693  0.081800  0.076435  1.000000  0.078711   
...              ...       ...       ...       ...       ...       ...   
97928       0.004515  0.004521  0.004543  0.004526  0.004530  0.004533   
98069       0.004583  0.004614  0.004595  0.004578  0.004578  0.004578   
98159       0.004355  0.004367  0.004365  0.004365  0.004367  0.004363   
98185       0.004167  0.004166  0.004179  0.004175  0.004177  0.004167   
98200       0.004333  0.004335  0.004333  0.004339  0.004349  0.004347   

CustomerID     477       639       649       669    ...     97697     97753  \
CustomerID                                          ...                       
33          0.074573  0.083020  0.081503  0.080070  ...  0.004811  0.004669   
200         0.075956  0.076435  0.077674  0.076923  ...  0.004824  0.004681   
264         0.076923  0.080634  0.082100  0.078448  ...  0.004822  0.004674   
356         0.075025  0.082403  0.077171  0.075956  ...  0.004816  0.004671   
412         0.075025  0.082403  0.078187  0.078448  ...  0.004810  0.004702   
...              ...       ...       ...       ...  ...       ...       ...   
97928       0.004534  0.004526  0.004528  0.004523  ...  0.003509  0.003509   
98069       0.004589  0.004581  0.004585  0.004592  ...  0.003509  0.003509   
98159       0.004366  0.004363  0.004369  0.004365  ...  0.003509  0.003565   
98185       0.004168  0.004176  0.004172  0.004161  ...  0.003311  0.003162   
98200       0.004338  0.004335  0.004329  0.004347  ...  0.003418  0.003345   

CustomerID     97769     97793     97900     97928     98069     98159  \
CustomerID                                                               
33          0.004412  0.005019  0.004312  0.004515  0.004583  0.004355   
200         0.004431  0.005047  0.004311  0.004521  0.004614  0.004367   
264         0.004416  0.005035  0.004322  0.004543  0.004595  0.004365   
356         0.004416  0.005038  0.004310  0.004526  0.004578  0.004365   
412         0.004414  0.005034  0.004318  0.004530  0.004578  0.004367   
...              ...       ...       ...       ...       ...       ...   
97928       0.003300  0.003653  0.003579  1.000000  0.003638  0.003418   
98069       0.003369  0.003509  0.003496  0.003638  1.000000  0.003469   
98159       0.003322  0.003431  0.003322  0.003418  0.003469  1.000000   
98185       0.003255  0.003406  0.003172  0.003255  0.003345  0.003277   
98200       0.003381  0.003444  0.003288  0.003245  0.003381  0.003266   

CustomerID     98185     98200  
CustomerID                      
33          0.004167  0.004333  
200         0.004166  0.004335  
264         0.004179  0.004333  
356         0.004175  0.004339  
412         0.004177  0.004349  
...              ...       ...  
97928       0.003255  0.003245  
98069       0.003345  0.003381  
98159       0.003277  0.003266  
98185       1.000000  0.003288  
98200       0.003288  1.000000  

[1000 rows x 1000 columns]

## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [24]:
similarities = distances[33].sort_values(ascending=False)[1:6]
similarities

CustomerID
264     0.087047
3535    0.087047
3317    0.087047
2503    0.085983
3305    0.085638
Name: 33, dtype: float64

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [96]:
sim = list(similarities.index)
sim_customers = quantities.loc[sim]
sim_customers

Int64Index([18027, 18281, 17136, 13681, 5224], dtype='int64', name='CustomerID')


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [68]:
ranked = sim_customers.groupby(['ProductName']).agg({'Quantity':'sum'}).sort_values(by='Quantity', ascending=False)
ranked

Quantity
ProductName                            
Butter - Unsalted                     3
Wine - Ej Gallo Sierra Valley         3
Towels - Paper / Kraft                3
Soup - Campbells Bean Medley          3
Wine - Blue Nun Qualitatswein         3
...                                 ...
Hinge W Undercut                      1
Ice Cream Bar - Hageen Daz To         1
Jagermeister                          1
Jolt Cola - Electric Blue             1
Yogurt - French Vanilla               1

[218 rows x 1 columns]

## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [71]:
merged = ranked.merge(table[33], on='ProductName')
merged

Quantity   33
ProductName                                 
Butter - Unsalted                     3  0.0
Wine - Ej Gallo Sierra Valley         3  0.0
Towels - Paper / Kraft                3  1.0
Soup - Campbells Bean Medley          3  0.0
Wine - Blue Nun Qualitatswein         3  0.0
...                                 ...  ...
Hinge W Undercut                      1  0.0
Ice Cream Bar - Hageen Daz To         1  0.0
Jagermeister                          1  0.0
Jolt Cola - Electric Blue             1  0.0
Yogurt - French Vanilla               1  1.0

[218 rows x 2 columns]

In [82]:
not_purchased = merged.loc[merged[33] == 0.0]
not_purchased.head()

Quantity   33
ProductName                                 
Butter - Unsalted                     3  0.0
Wine - Ej Gallo Sierra Valley         3  0.0
Soup - Campbells Bean Medley          3  0.0
Wine - Blue Nun Qualitatswein         3  0.0
Chicken - Soup Base                   2  0.0

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [126]:
recommendations = {}
customers = set(list(data['CustomerID']))
len(customers)
for customer in customers:
    sims = distances[customer].sort_values(ascending=False)[1:6]
    s = list(sims.index)
    sims_cust = quantities.loc[s]
    rank = sims_cust.reset_index().groupby(['ProductName']).agg({'Quantity':'sum'}).sort_values(by='Quantity', ascending=False)
    mergeo = rank.merge(table[customer], on='ProductName')
    filtered = mergeo.loc[mergeo[customer]==0.0].sort_values(by='Quantity', ascending=False)[1:6]
    what = list(filtered.index)
    recommendations[customer] = what
    
#recommendations

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [128]:
result = pd.DataFrame(recommendations)
result = result.T
result

0                             1  \
83973               Veal - Slab Bacon         Bandage - Fexible 1x3   
59399          Towels - Paper / Kraft         Knife Plastic - White   
92168               Snapple Lemon Tea  Veal - Brisket, Provimi,bnls   
49159              Cookies - Assorted                 Puree - Mocha   
18441       Beef - Texas Style Burger     Anchovy Paste - 56 G Tube   
...                               ...                           ...   
77814       Macaroons - Two Bite Choc             Tilapia - Fillets   
32753                    Sherry - Dry      Chestnuts - Whole,canned   
20476  Pasta - Detalini, White, Fresh               Skirt - 29 Foot   
67582           Ecolab - Solid Fusion       Wine - Redchard Merritt   
65535        Chestnuts - Whole,canned          Scallop - St. Jaques   

                                    2                            3  \
83973            Ezy Change Mophandle            Chicken - Wieners   
59399    Nantucket - Pomegranate Pear     Snapple - Iced Tea Peach   
92168                  Spinach - Baby        Pop Shoppe Cream Soda   
49159        Cheese - Parmesan Grated  Cheese - Victor Et Berthold   
18441                  Flour - Pastry                Foam Cup 6 Oz   
...                               ...                          ...   
77814          Wine - Ruffino Chianti   Cake - Cake Sheet Macaroon   
32753         Napkin White - Starched            Sprouts - Alfalfa   
20476                     Wanton Wrap                 Broom - Corn   
67582                Mussels - Frozen          Cheese - Bocconcini   
65535  Bread - Roll, Soft White Round            Chicken - Wieners   

                                   4  
83973          Puree - Passion Fruit  
59399     Cookie Chocolate Chip With  
92168     Bread - Calabrese Baguette  
49159        Chicken - Leg, Boneless  
18441       Garlic - Primerba, Paste  
...                              ...  
77814                    Guinea Fowl  
32753      Appetizer - Sausage Rolls  
20476  Ice Cream Bar - Hageen Daz To  
67582             Peas - Pigeon, Dry  
65535           Lamb - Pieces, Diced  

[1000 rows x 5 columns]

## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [134]:
squareform(pdist(table.T, 'cosine'))
distances = pd.DataFrame(1/(1 + squareform(pdist(table.T, 'cosine'))), index=table.columns, columns=table.columns)

In [133]:
recommendations1 = {}
customers1 = set(list(data['CustomerID']))
len(customers1)
for customers in customers1:
    sims1 = distances[customers].sort_values(ascending=False)[1:6]
    s1 = list(sims1.index)
    sims_cust1 = quantities.loc[s1]
    rank1 = sims_cust1.reset_index().groupby(['ProductName']).agg({'Quantity':'sum'}).sort_values(by='Quantity', ascending=False)
    mergeo1 = rank1.merge(table[customers], on='ProductName')
    filtered1 = mergeo1.loc[mergeo1[customers]==0.0].sort_values(by='Quantity', ascending=False)[1:6]
    what1 = list(filtered1.index)
    recommendations1[customers] = what1
#recommendations1

In [132]:
result1 = pd.DataFrame(recommendations1)
result1 = result1.T
result1

0                                  1  \
83973                Flour - Pastry              Puree - Passion Fruit   
59399     Anchovy Paste - 56 G Tube                    Garlic - Peeled   
92168           Tuna - Salad Premix                      Sage - Ground   
49159                   Beer - Blue                      Peas - Frozen   
18441  Whmis - Spray Bottle Trigger               Mussels - Cultivated   
...                             ...                                ...   
77814      Scallops - Live In Shell                           Remy Red   
32753       Mushroom - Porcini, Dry              Rice - Jasmine Sented   
20476                  Onion Powder  Beer - Alexander Kieths, Pale Ale   
67582       Crab - Imitation Flakes          Tea - Herbal Sweet Dreams   
65535          Coffee - Irish Cream   Muffin Chocolate Individual Wrap   

                                   2                         3  \
83973           Lettuce - Spring Mix              Broom - Corn   
59399                   Broom - Corn  Water - Aquafina Vitamin   
92168                 Apricots Fresh       Beef - Inside Round   
49159                  Lamb - Ground      Beef - Ground Medium   
18441  Ecolab - Lime - A - Way 4/4 L                 Milk - 2%   
...                              ...                       ...   
77814      Beef - Texas Style Burger      Bread Crumbs - Panko   
32753            Tuna - Salad Premix      Beef - Ground Medium   
20476        Turnip - White, Organic       Cheese Cloth No 100   
67582                  Cumin - Whole            Flour - Pastry   
65535      Coconut - Shredded, Sweet              Blackberries   

                                  4  
83973          Cheese - Brie,danish  
59399        Cake - Mini Cheesecake  
92168  Nantucket - Pomegranate Pear  
49159        Beef - Chuck, Boneless  
18441            Tea - Jasmin Green  
...                             ...  
77814          Curry Paste - Madras  
32753          Sun - Dried Tomatoes  
20476      Tomato - Tricolor Cherry  
67582       Pastry - Choclate Baked  
65535       Turnip - White, Organic  

[1000 rows x 5 columns]